## Model Evaluation Trial-runs 

In [ ]:
import os 

In [ ]:
chdir()

## Entity Trial-runs 

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path 
    metric_file_name: Path

## ConfigurationManager Trial-runs 

In [ ]:
#importing all project paths and modules necessary for project configurations 
from textSummerizer.constants import  *
from textSummerizer.utils.common import read_yaml, create_directories

#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation 
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path, 
            metric_file_name=config.metric_file_name
            
        )
        return model_evaluation_config

## Components Trial-runs 

In [ ]:
#Importing labraries 
from transfomers import AutoModelForSeq25eqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric 
import torch
import pandas as pd
from tqdm import tqdm  

In [ ]:
class ModelEvaluation: 
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size): 
        """Split the dataset into smaller batches that we can process simultaneously
        Yeild successive batched sized chunks from list_of_elements
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
        batch_size=16, device="cuba" if torch.cuda.is_vailable() else "cpu", 
        column_text="article", column_summary="highlights"):

        article_batches=list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches=list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches),
            total=len(article_batches)
        ):

        inputs = tokenizer(article_batch, max_length=1024, 
            truncation=True,padding="max_length", return_tensors="pt")

        summaries= model.generate(
            input_ids=inputs["input_ids"].to(device), 
            attention_mask=inputs["attention_mask"].to(device),
            length_penalt=0.8,#ensures model does not generate sequences lower  
            num_beans=8,
            max_length=128
        )

            #Finally we decode the gerated text 
            #replace the token, and add decoded texts with reference to the metric 
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                clean_up_tokenization_spaces=True) for s in summaries]

                metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
            #Finally compute and return GOUGE scores 
            score= metric.compute()
            return score


    def evaluate(self): 
        device = "cuba" if torch.cuda.is_vailable() else "cpu"
        tokenizer = AutoTokenizer.from_predicted(self.config.tokenizer_path)
        model_pegasus= AutoModelSeq25eqLM.from_predicted(self.config.model_path).to(device) 

        #Loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougel", "rougelsum"]

        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10],
            rouge_metric,
            model_pegasus,
            tokenizer,
            batch_size=2,
            column_text=,

        )
        
        rouge_dict =dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)


## Pipeline Trial-runs 

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config =config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e